# Методы сбора и обработки данных из сети Интернет
## Системы управления базами данных MongoDB и SQLite в Python

Загрузка необходимых библиотек

In [1]:
import requests
from bs4 import BeautifulSoup
import re

import tqdm.notebook
import pandas as pd
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke, BulkWriteError as bwe

Парсинг сайта

In [2]:
base_url = 'https://hh.ru'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36'}

In [3]:
def parse_salary(salary):
    salary = salary.replace(" ", '').replace(' ', '')
    res = [None, None, None]
    
    try:
        match_res = re.match('(\d+).(\d+)(.*)', salary) # dual salary
        res[0] = int(match_res.group(1))
        res[1] = int(match_res.group(2))
        res[2] = match_res.group(3)
    except:
        pass
    
    try:
        match_res = re.match('от(\d+)(.*)', salary) # from-salary
        res[0] = int(match_res.group(1))
        res[2] = match_res.group(2)
    except:
        pass
    
    try:
        match_res = re.match('до(\d+)(.*)', salary) # to-salary
        res[1] = int(match_res.group(1))
        res[2] = match_res.group(2)
    except:
        pass
    
    return res

In [4]:
jobs_list = []
page_number = int(input('Введите количество страниц, которые нужно обработать: '))
a = tqdm.notebook.tqdm(total=page_number)
job_title = input('Введите вакансию: ')

for page in range(page_number):
    params = {'text': job_title, 'page': page}
    response = requests.get(base_url + '/search/vacancy', headers=headers, params=params)
    dom = BeautifulSoup(response.text, 'html.parser')
    
    items = dom.find_all('div', {'class': 'vacancy-serp-item'})
    
    for item in items:
        salary = None
        title = None
        
        title = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
        
        try:
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            parsed_salary = parse_salary(salary)

        except AttributeError as e:
            parsed_salary = [None, None, None]
        
        link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
        vac_id = int(re.match('.*?(\d+).*?', link).groups(1)[0])
    
        jobs_list.append([vac_id, title, *parsed_salary, link, base_url])
    
    a.update(1)
    
    if len(dom.find_all('a', {'data-qa': 'pager-next'})) == 0:
        break
    


Введите количество страниц, которые нужно обработать: 30


  0%|          | 0/30 [00:00<?, ?it/s]

Введите вакансию: аналитик


In [5]:
jobs_df = pd.DataFrame(jobs_list, columns=['_id', 'Наименование вакансии', 'Минимальная зарплата', 'Максимальная зарплата', 'Валюта', 'Ссылка на вакансию', 'Ссылка на сайт'])
jobs_df

,_id,Наименование вакансии,Минимальная зарплата,Максимальная зарплата,Валюта,Ссылка на вакансию,Ссылка на сайт
0,53032262,Data-аналитик (Анализ цены),NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/53032262?fr...,https://hh.ru
1,46241509,Аналитик данных,NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/46241509?fr...,https://hh.ru
2,51907042,Экономист-аналитик,100000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/51907042?fr...,https://hh.ru
3,52030785,Data Analyst,NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/52030785?fr...,https://hh.ru
4,49881796,"Data Analyst \ Аналитик данных (BI, SQL, Pytho...",170000.0,210000.0,руб.,https://ekaterinburg.hh.ru/vacancy/49881796?fr...,https://hh.ru
...,...,...,...,...,...,...,...
595,52429718,Маркетолог-аналитик,NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/52429718?fr...,https://hh.ru
596,53076625,Аналитик отдела продаж,40000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/53076625?fr...,https://hh.ru
597,52226546,Аналитик информационных систем,NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/52226546?fr...,https://hh.ru
598,51346372,Бизнес – аналитик,60000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/51346372?fr...,https://hh.ru


### Задание 1
Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [6]:
client = MongoClient('localhost', 27017)

db = client['vacancies_hh']    
vacancies = db.vacancies        

In [7]:
try:
     vacancies.insert_many(jobs_df.drop_duplicates(subset = ['_id'], keep = 'first').to_dict('records'), ordered=False)
except (dke, bwe):
    print('Duplicate key error collection')

### Задание 2
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). 

In [9]:
salary_vac = int(input('Введите нужную зарплату: '))

for doc in vacancies.find({'$or': [{'Минимальная зарплата': {'$gte': salary_vac}},
                                   {'Максимальная зарплата': {'$gte': salary_vac}}]}):
     pprint(doc)

Введите нужную зарплату: 400000
{'_id': 52987990,
 'Валюта': 'KZT',
 'Максимальная зарплата': nan,
 'Минимальная зарплата': 500000.0,
 'Наименование вакансии': 'Финансовый аналитик в г. Шымкент (Аналитик продаж)',
 'Ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/52987990?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
 'Ссылка на сайт': 'https://hh.ru'}
{'_id': 53001435,
 'Валюта': 'руб.',
 'Максимальная зарплата': 530000.0,
 'Минимальная зарплата': 310000.0,
 'Наименование вакансии': 'Системный аналитик (Middle & Senior)',
 'Ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/53001435?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
 'Ссылка на сайт': 'https://hh.ru'}
{'_id': 52999639,
 'Валюта': 'руб.',
 'Максимальная зарплата': 420000.0,
 'Минимальная зарплата': nan,
 'Наименование вакансии': 'Системный аналитик',
 'Ссылка на вакансию': 'https:/